In [1]:
import cv2
from ultralytics import YOLO
import streamlink

In [5]:
import cv2
import time
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
# model = YOLO("yolov8n-seg.pt")
# model = YOLO("yolov8n-pose.pt")



# Open the video file
video_path = "D:\\Downloads\\traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Variables for FPS calculation
prev_frame_time = 0
new_frame_time = 0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,classes=0)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Calculate FPS
        new_frame_time = time.time()
        fps = 1.0 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time

        # Count the number of detected objects
        num_objects = len(results[0].boxes)

        # Put the FPS and object count on the frame
        cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Detected: {num_objects}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Put the "NUS-ISS Demo" text in the top-right corner
        text = "NUS-ISS Demo"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_x = annotated_frame.shape[1] - text_size[0] - 10
        text_y = 30
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            print("test")
            break
    else:
        # Break the loop if the end of the video is reached
        print("test")
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 92.0ms
Speed: 3.0ms preprocess, 92.0ms inference, 64.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 64

In [6]:
import cv2
import time
from ultralytics import YOLO
import numpy as np
import streamlink

# Load the YOLOv8 model
# model = YOLO("yolov8n.pt")
# model = YOLO("yolov8l.pt")
# model = YOLO("yolov8n-obb.pt")
# model = YOLO("yolov8n-seg.pt")
# model = YOLO("yolov8l-seg.pt")
# model = YOLO("mybest.pt")
model = YOLO("hatbest.pt")

# Open Live stream
# url = 'https://www.youtube.com/watch?v=DjdUEyjx8GM'
url = 'https://www.youtube.com/watch?v=gFRtAAmiFbE'
# url = 'https://www.youtube.com/watch?v=KY4Yd5QR570'

# streams = streamlink.streams(url)
# streams = streamlink.streams(url)
# cap = cv2.VideoCapture(streams["720p"].url)
# cap = cv2.VideoCapture(streams["best"].url)

# Open the video file
video_path = "short_video.mp4"
cap = cv2.VideoCapture(video_path)

# Variables for FPS calculation
prev_frame_time = 0
new_frame_time = 0

# Get the video frame dimensions
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the counting lines (adjust these values as needed)
left_line = int(frame_width * 0.2)  # 20% from the left
right_line = int(frame_width * 0.8)  # 20% from the right

# Counters for people exiting
left_exit_count = 0
right_exit_count = 0

# Dictionary to store track histories
track_history = {}

# Sets to store IDs of people who have left
left_exited_ids = set()
right_exited_ids = set()

# Function to check if a person has crossed a line
def has_crossed_line(prev_pos, curr_pos, line_pos):
    return (prev_pos < line_pos and curr_pos >= line_pos) or (prev_pos > line_pos and curr_pos <= line_pos)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, conf=0.1)  # class 0 is person
        
        # Visualize the results on the frame
        annotated_frame = results[0].plot() # render annotated frame
        # annotated_frame = frame.copy() # render empty frame

        # Process each detected person
        if results[0].masks is not None:
            for mask in results[0].masks.xy:
                # Convert the polygon to a format suitable for cv2.polylines
                polygon = np.array(mask, dtype=np.int32)
                
                # Draw the polygon on the frame
                # cv2.polylines(annotated_frame, [polygon], isClosed=True, color=(0, 0, 255), thickness=2)
                cv2.fillPoly(annotated_frame, [polygon], color=(255, 0, 0, 127))  # semi-transparent fill
        

        # Draw the counting lines
        cv2.line(annotated_frame, (left_line, 0), (left_line, frame_height), (255, 255, 0), 2)
        cv2.line(annotated_frame, (right_line, 0), (right_line, frame_height), (255, 255, 0), 2)
        
        # Process each detected person
        if results[0].boxes is not None and results[0].boxes.id is not None:
            for box, track_id in zip(results[0].boxes.xywh, results[0].boxes.id):
                x, y, w, h = box
                track_id = int(track_id)
                center_x, center_y = int(x), int(y)
                
                # Store the center points of each track
                if track_id not in track_history:
                    track_history[track_id] = []
                track_history[track_id].append((center_x, center_y))
                
                # Keep only the last 30 positions
                track_history[track_id] = track_history[track_id][-30:]
                
                # Check if the person has crossed a line, algo sucks....
                if len(track_history[track_id]) > 1:
                    prev_x = np.mean([pos[0] for pos in track_history[track_id][:-10]])
                    curr_x = np.mean([pos[0] for pos in track_history[track_id][-10:]])
                    
                    if has_crossed_line(prev_x, curr_x, left_line) and track_id not in left_exited_ids:
                        left_exit_count += 1
                        left_exited_ids.add(track_id)
                    elif has_crossed_line(prev_x, curr_x, right_line) and track_id not in right_exited_ids:
                        right_exit_count += 1
                        right_exited_ids.add(track_id)
                
                # Draw the track
                if len(track_history[track_id]) > 1:
                    cv2.polylines(annotated_frame, [np.array(track_history[track_id], dtype=np.int32)], False, (0, 255, 0), 2)
                
                # Draw ID near the person
                cv2.putText(annotated_frame, f"ID: {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Calculate FPS
        new_frame_time = time.time()
        fps = 1.0 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        
        # Count the number of detected objects
        num_objects = len(results[0].boxes) if results[0].boxes is not None else 0
        
        # Put the FPS, object count, and exit counts on the frame
        cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Detected: {num_objects}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Left exits: {left_exit_count}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Right exits: {right_exit_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Put the "NUS-ISS Demo" text in the top-right corner
        text = "NUS-ISS Demo"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_x = annotated_frame.shape[1] - text_size[0] - 10
        text_y = 30
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 2 heads, 1 helmet, 1 person, 29.0ms
Speed: 0.0ms preprocess, 29.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 heads, 2 persons, 2.0ms
Speed: 2.6ms preprocess, 2.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 heads, 2 persons, 12.8ms
Speed: 0.0ms preprocess, 12.8ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 heads, 2 persons, 5.7ms
Speed: 0.0ms preprocess, 5.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 heads, 8.3ms
Speed: 0.0ms preprocess, 8.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 8.2ms
Speed: 1.8ms preprocess, 8.2ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 10.2ms
Speed: 0.0ms preprocess, 10.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postproc

In [18]:
# masks[0].xy

In [7]:
import cv2
import time
from ultralytics import YOLO
import numpy as np
import streamlink

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
# model = YOLO("yolov8l.pt")
# model = YOLO("yolov8x.pt")
# model = YOLO("yolov8n-obb.pt")
# model = YOLO("yolov8n-seg.pt")
# model = YOLO("yolov8l-seg.pt")
# model = YOLO("mybest.pt")
# model = YOLO("hatbest.pt")
# model = YOLO("my-best-segment.pt")
# model = YOLO("yolov10n.pt")

# Open Live stream
url = 'https://www.youtube.com/watch?v=DjdUEyjx8GM'
# url = 'https://www.youtube.com/watch?v=gFRtAAmiFbE'
# url = 'https://www.youtube.com/watch?v=KY4Yd5QR570'

# streams = streamlink.streams(url)
streams = streamlink.streams(url)
cap = cv2.VideoCapture(streams["720p"].url)
# cap = cv2.VideoCapture(streams["best"].url)

# Open the video file
# video_path = "short_video.mp4"
# cap = cv2.VideoCapture(video_path)

# Variables for FPS calculation
prev_frame_time = 0
new_frame_time = 0

# Get the video frame dimensions
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the counting lines (adjust these values as needed)
left_line = int(frame_width * 0.2)  # 20% from the left
right_line = int(frame_width * 0.8)  # 20% from the right

# Counters for people exiting
left_exit_count = 0
right_exit_count = 0

# Dictionary to store track histories
track_history = {}

# Sets to store IDs of people who have left
left_exited_ids = set()
right_exited_ids = set()

# Function to check if a person has crossed a line
def has_crossed_line(prev_pos, curr_pos, line_pos):
    return (prev_pos < line_pos and curr_pos >= line_pos) or (prev_pos > line_pos and curr_pos <= line_pos)

def smooth_polygon(polygon, smoothing=5):
    """Apply smoothing to the polygon points."""
    smooth_polygon = np.array(polygon, dtype=np.float32)
    for _ in range(smoothing):
        smooth_polygon = np.array([np.mean(np.roll(smooth_polygon, shift, axis=0), axis=0)
                                   for shift in range(-1, 2)])
    return np.array(smooth_polygon, dtype=np.int32)


# Frame skip variable
frame_skip = 0 # 2 is good
frame_counter = 0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    # Skip frames to catch up with the stream
    # frame_counter += 1
    # if frame_skip > 0:
    #     if frame_counter % frame_skip != 0:
    #         continue
    
    if success:
        
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        # results = model.track(frame, persist=True, classes=0)  # class 0 is person
        results = model.track(frame, persist=True, classes=0, conf=0.1,tracker="bytetrack.yaml")  # class 0 is person, conf=0.25 sets the confidence threshold to 25%
        # results = model.track(frame, persist=True, classes=0, conf=0.1)  # class 0 is person, conf=0.25 sets the confidence threshold to 25%
        # results = model.track(frame, persist=True, conf=0.1)
        # annotation
        # frame = results[0].plot() # info from yolo v8, optional, can comment off
        
        # # Create a blank overlay for the semi-transparent masks
        overlay = np.zeros_like(frame, dtype=np.uint8)
        
                                    # Process each detected person
        if results[0].masks is not None:
            for mask in results[0].masks.xy:
                # Convert the polygon to a format suitable for cv2.fillPoly
                polygon = np.array(mask, dtype=np.int32)
                
                # Fill the polygon on the overlay
                cv2.fillPoly(overlay, [polygon], color=(255, 0, 255))  # Red color
        
        # Blend the overlay with the original frame
        alpha = 0.3  # Adjust this value to change the transparency (0.0 - 1.0)
        annotated_frame = cv2.addWeighted(frame, 1, overlay, alpha, 0)

        # Draw the counting lines
        cv2.line(annotated_frame, (left_line, 0), (left_line, frame_height), (255, 255, 0), 2)
        cv2.line(annotated_frame, (right_line, 0), (right_line, frame_height), (255, 255, 0), 2)
        
        # Process each detected person
        if results[0].boxes is not None and results[0].boxes.id is not None:
            for box, track_id in zip(results[0].boxes.xywh, results[0].boxes.id):
                x, y, w, h = box
                track_id = int(track_id)
                center_x, center_y = int(x), int(y)
                
                # Store the center points of each track
                if track_id not in track_history:
                    track_history[track_id] = []
                track_history[track_id].append((center_x, center_y))
                
                # Keep only the last 30 positions
                track_history[track_id] = track_history[track_id][-30:]
                
                # Check if the person has crossed a line, algo sucks....
                if len(track_history[track_id]) > 1:
                    prev_x = np.mean([pos[0] for pos in track_history[track_id][:-10]])
                    curr_x = np.mean([pos[0] for pos in track_history[track_id][-10:]])
                    
                    if has_crossed_line(prev_x, curr_x, left_line) and track_id not in left_exited_ids:
                        left_exit_count += 1
                        left_exited_ids.add(track_id)
                    elif has_crossed_line(prev_x, curr_x, right_line) and track_id not in right_exited_ids:
                        right_exit_count += 1
                        right_exited_ids.add(track_id)
                
                # Draw the track
                if len(track_history[track_id]) > 1:
                    cv2.polylines(annotated_frame, [np.array(track_history[track_id], dtype=np.int32)], False, (0, 255, 0), 2)
                
                # Draw ID near the person
                cv2.putText(annotated_frame, f"ID: {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Calculate FPS
        new_frame_time = time.time()
        fps = 1.0 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        
        # Count the number of detected objects
        num_objects = len(results[0].boxes) if results[0].boxes is not None else 0
        
        # Put the FPS, object count, and exit counts on the frame
        cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Detected: {num_objects}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Left exits: {left_exit_count}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Right exits: {right_exit_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Put the "NUS-ISS Demo" text in the top-right corner
        text = "NUS-ISS Demo"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_x = annotated_frame.shape[1] - text_size[0] - 10
        text_y = 30
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 5 persons, 42.0ms
Speed: 2.7ms preprocess, 42.0ms inference, 7.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8.4ms
Speed: 2.3ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 36.1ms
Speed: 0.0ms preprocess, 36.1ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 5.6ms
Speed: 2.4ms preprocess, 5.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6.0ms
Speed: 2.8ms preprocess, 6.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 7.8ms
Speed: 1.2ms preprocess, 7.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 6.4ms
Speed: 1.9ms preprocess, 6.4ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 5.4ms
Speed: 2.2ms preprocess, 5.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 6

c:\Users\i_kaz\anaconda3\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\i_kaz\anaconda3\envs\torch\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0: 384x640 7 persons, 9.0ms
Speed: 1.3ms preprocess, 9.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 5.8ms
Speed: 2.4ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 6.1ms
Speed: 1.4ms preprocess, 6.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 5.4ms
Speed: 1.9ms preprocess, 5.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 8.2ms
Speed: 0.0ms preprocess, 8.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 8.1ms
Speed: 0.0ms preprocess, 8.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 9.1ms
Speed: 0.0ms preprocess, 9.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 5.9ms
Speed: 4.4ms preprocess, 5.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



In [15]:
from collections import defaultdict

import cv2

from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import streamlink

url = 'https://www.youtube.com/watch?v=gFRtAAmiFbE'
streams = streamlink.streams(url)

track_history = defaultdict(lambda: [])

model = YOLO("yolov8n-seg.pt")  # segmentation model
cap = cv2.VideoCapture(streams["720p"].url)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# out = cv2.VideoWriter("instance-segmentation-object-tracking.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)

    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for mask, track_id in zip(masks, track_ids):
            color = colors(int(track_id), True)
            txt_color = annotator.get_txt_color(color)
            annotator.seg_bbox(mask=mask, mask_color=color, track_label=str(track_id))

    # out.write(im0)
    cv2.imshow("instance-segmentation-object-tracking", im0)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

out.release()
cap.release()
cv2.destroyAllWindows()


0: 384x640 28 persons, 1 traffic light, 2 backpacks, 1 umbrella, 2 handbags, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 16.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 persons, 1 traffic light, 2 backpacks, 1 umbrella, 2 handbags, 7.0ms
Speed: 63.9ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 persons, 1 traffic light, 2 backpacks, 1 umbrella, 2 handbags, 7.0ms
Speed: 1.4ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 persons, 1 traffic light, 2 backpacks, 1 umbrella, 2 handbags, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 27 persons, 1 traffic light, 2 backpacks, 1 umbrella, 3 handbags, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 persons, 1 traffic light, 2 backpacks, 1 umbrella, 3 handbags, 7.0ms
Speed: 1.0ms

In [12]:
help(annotator.seg_bbox)

Help on method seg_bbox in module ultralytics.utils.plotting:

seg_bbox(mask, mask_color=(255, 0, 255), det_label=None, track_label=None) method of ultralytics.utils.plotting.Annotator instance
    Function for drawing segmented object in bounding box shape.
    
    Args:
        mask (list): masks data list for instance segmentation area plotting
        mask_color (tuple): mask foreground color
        det_label (str): Detection label text
        track_label (str): Tracking label text



In [12]:
import cv2
import time
from ultralytics import YOLO
import numpy as np
import streamlink

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
# model = YOLO("yolov8l.pt")
# model = YOLO("yolov8n-obb.pt")
model = YOLO("yolov8n-seg.pt")
# model = YOLO("yolov8l-seg.pt")
model = YOLO("mybest.pt")
model = YOLO("hatbest.pt")

# Open the video file
video_path = "D:\\Downloads\\traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Variables for FPS calculation
prev_frame_time = 0
new_frame_time = 0

# Get the video frame dimensions
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the counting lines (adjust these values as needed)
left_line = int(frame_width * 0.2)  # 20% from the left
right_line = int(frame_width * 0.8)  # 20% from the right

# Counters for people exiting
left_exit_count = 0
right_exit_count = 0

# Dictionary to store track histories
track_history = {}

# Sets to store IDs of people who have left
left_exited_ids = set()
right_exited_ids = set()

# Function to check if a person has crossed a line
def has_crossed_line(prev_pos, curr_pos, line_pos):
    return (prev_pos < line_pos and curr_pos >= line_pos) or (prev_pos > line_pos and curr_pos <= line_pos)

def smooth_polygon(polygon, smoothing=5):
    """Apply smoothing to the polygon points."""
    smooth_polygon = np.array(polygon, dtype=np.float32)
    for _ in range(smoothing):
        smooth_polygon = np.array([np.mean(np.roll(smooth_polygon, shift, axis=0), axis=0)
                                   for shift in range(-1, 2)])
    return np.array(smooth_polygon, dtype=np.int32)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    
    if success:
        
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        # results = model.track(frame, persist=True, classes=0)  # class 0 is person
        results = model.track(frame, persist=True, classes=0, conf=0.1)  # class 0 is person, conf=0.25 sets the confidence threshold to 25%

        # # Create a blank overlay for the semi-transparent masks
        overlay = np.zeros_like(frame, dtype=np.uint8)
        
        # Process each detected person
        if results[0].masks is not None:
            for mask in results[0].masks.xy:
                # Convert the polygon to a format suitable for cv2.fillPoly
                polygon = np.array(mask, dtype=np.int32)
                
                # Fill the polygon on the overlay
                cv2.fillPoly(overlay, [polygon], color=(255, 0, 255))  # Red color
        
        # Blend the overlay with the original frame
        alpha = 0.3  # Adjust this value to change the transparency (0.0 - 1.0)
        annotated_frame = cv2.addWeighted(frame, 1, overlay, alpha, 0)

        # Draw the counting lines
        cv2.line(annotated_frame, (left_line, 0), (left_line, frame_height), (255, 255, 0), 2)
        cv2.line(annotated_frame, (right_line, 0), (right_line, frame_height), (255, 255, 0), 2)
        
        # Process each detected person
        if results[0].boxes is not None and results[0].boxes.id is not None:
            for box, track_id in zip(results[0].boxes.xywh, results[0].boxes.id):
                x, y, w, h = box
                track_id = int(track_id)
                center_x, center_y = int(x), int(y)
                
                # Store the center points of each track
                if track_id not in track_history:
                    track_history[track_id] = []
                track_history[track_id].append((center_x, center_y))
                
                # Keep only the last 30 positions
                track_history[track_id] = track_history[track_id][-30:]
                
                # Check if the person has crossed a line, algo sucks....
                if len(track_history[track_id]) > 1:
                    prev_x = np.mean([pos[0] for pos in track_history[track_id][:-10]])
                    curr_x = np.mean([pos[0] for pos in track_history[track_id][-10:]])
                    
                    if has_crossed_line(prev_x, curr_x, left_line) and track_id not in left_exited_ids:
                        left_exit_count += 1
                        left_exited_ids.add(track_id)
                    elif has_crossed_line(prev_x, curr_x, right_line) and track_id not in right_exited_ids:
                        right_exit_count += 1
                        right_exited_ids.add(track_id)
                
                # Draw the track
                if len(track_history[track_id]) > 1:
                    cv2.polylines(annotated_frame, [np.array(track_history[track_id], dtype=np.int32)], False, (0, 255, 0), 2)
                
                # Draw ID near the person
                cv2.putText(annotated_frame, f"ID: {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Calculate FPS
        new_frame_time = time.time()
        fps = 1.0 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        
        # Count the number of detected objects
        num_objects = len(results[0].boxes) if results[0].boxes is not None else 0
        
        # Put the FPS, object count, and exit counts on the frame
        cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Detected: {num_objects}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Left exits: {left_exit_count}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"Right exits: {right_exit_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Put the "NUS-ISS Demo" text in the top-right corner
        text = "NUS-ISS Demo"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_x = annotated_frame.shape[1] - text_size[0] - 10
        text_y = 30
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [26]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
model = YOLO("yolov8n-seg")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="coco8-seg.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.72 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.50  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=coco8-seg.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, 

100%|███████████████████████████████████████████████████████████████████████████████| 439k/439k [00:00<00:00, 32.1MB/s]
Unzipping C:\Users\i_kaz\datasets\coco8-seg.zip to C:\Users\i_kaz\datasets\coco8-seg...: 100%|██████████| 25/25 [00:00<

Dataset download success  (2.8s), saved to C:\Users\i_kaz\datasets


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                

YOLOv8n-seg summary: 261 layers, 3409968 parameters, 3409952 gradients, 12.8 GFLOPs

Transferred 417/417 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs\segment\train2', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed 


train: Scanning C:\Users\i_kaz\datasets\coco8-seg\labels\train... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 

...

                   all          4         17      0.913       0.38      0.511      0.309      0.778       0.28       0.37      0.242



100 epochs completed in 0.026 hours.
Optimizer stripped from runs\segment\train2\weights\last.pt, 7.1MB
Optimizer stripped from runs\segment\train2\weights\best.pt, 7.1MB

Validating runs\segment\train2\weights\best.pt...
Ultralytics YOLOv8.2.50  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all          4         17      0.696      0.617      0.893       0.62      0.684        0.6      0.854       0.53
                person          3         10      0.512        0.7      0.681      0.318      0.439        0.6      0.618      0.253
                   dog          1          1      0.807          1      0.995      0.895      0.807          1      0.995      0.895
                 horse          1          2      0.305          1      0.995      0.598      0.305          1      0.995      0.324
              elephant          1          2          1          0      0.695      0.366          1          0      0.523      0.314
              umbrella          1          1      0.551          1      0.995      0.895      0.551          1      0.995      0.895
          potted plant          1          1          1          0      0.995      0.647          1          0      0.995      0.497
Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 1.2ms postproce

lr/pg0,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
lr/pg1,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
lr/pg2,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
metrics/mAP50(B),▆▆▆▇████▇▇▇▇▇▇▇▄▃▃▃▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂█
metrics/mAP50(M),▅▅▅▇██▇██████▇▇▄▃▃▄▄▃▃▃▃▃▃▃▁▃▃▁▁▂▃▃▂▂▂▃█
metrics/mAP50-95(B),▆▆▆▇▇█▇▇▇▇▇▇▇▆▆▄▃▃▃▂▂▂▂▁▂▁▁▁▂▂▁▁▁▂▂▁▁▁▂█
metrics/mAP50-95(M),▆▆▆▇██▇▇▇▇███▇▇▄▃▃▃▂▂▂▂▁▂▂▂▁▂▂▁▁▁▂▂▁▁▁▂█
metrics/precision(B),▂▂▂▂▂▂▂▂▁▁▁████████████▇█▇▇▇▇▇██▇▇▇▇▇▇▇▂
metrics/precision(M),▂▂▁▂▂▂▂▂▁▁▂████▆▄▄▇▇▇▆▆▇▆▆▆▄▆▆▄▄▄▆▆▄▄▄▄▂
metrics/recall(B),███████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
metrics/recall(M),▇▆▅▆█▆▆▆▆▆▆▃▃▃▃▁▂▂▃▂▂▃▃▃▃▂▂▂▂▂▂▁▁▂▂▁▁▁▁▆


In [14]:
help(model.track)

Help on method track in module ultralytics.engine.model:

track(source: Union[str, pathlib.Path, int, list, tuple, numpy.ndarray, torch.Tensor] = None, stream: bool = False, persist: bool = False, **kwargs) -> List[ultralytics.engine.results.Results] method of ultralytics.models.yolo.model.YOLO instance
    Conducts object tracking on the specified input source using the registered trackers.
    
    This method performs object tracking using the model's predictors and optionally registered trackers. It is
    capable of handling different types of input sources such as file paths or video streams. The method supports
    customization of the tracking process through various keyword arguments. It registers trackers if they are not
    already present and optionally persists them based on the 'persist' flag.
    
    The method sets a default confidence threshold specifically for ByteTrack-based tracking, which requires low
    confidence predictions as input. The tracking mode is expli